# Lab 2: Дерево рішень та ліс дерев рішень

In [113]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Dataset #1: Emails

In [117]:
import re
from nltk.corpus import stopwords
import string

df = pd.DataFrame(pd.read_csv('Phishing_Email.csv')) # https://www.kaggle.com/datasets/subhajournal/phishingemails
df.dropna(inplace=True)
df.drop(columns=df.columns[0], inplace=True)
def process_text(transformer: str) -> str:
    transformer = transformer.lower()
    transformer = re.sub(r"https*\S+", " ", transformer)
    transformer = re.sub(r"^([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-]+)(\.[a-zA-Z]{2,5}){1,2}$", " ", transformer)
    transformer = re.sub("[%s]" % re.escape(string.punctuation), " ", transformer)
    transformer = re.sub(r"\s{2,}", " ", transformer)
    
    result = ' '.join([word for word in transformer.split(' ') if word not in stopwords.words("english")])
    return result.strip()

x = df.drop('Email Type', axis=1)['Email Text'].apply(process_text)
y = LabelEncoder().fit_transform(df['Email Type'])


x_train_email, x_test_email, y_train_email, y_test_email = train_test_split(x, y, test_size=0.2, random_state=0)

## Векторизація тексту за TF-IDF та Count Vectorizer

In [118]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cv = CountVectorizer(max_features=1000, ngram_range=(1, 2))
tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

x_train_vectorised_tfidf = tfidf.fit_transform(x_train_email)
x_test_vectorised_tfidf = tfidf.transform(x_test_email)
x_train_vectorised_cv = cv.fit_transform(x_train_email)
x_test_vectorised_cv = cv.transform(x_test_email)

## Decision Tree and Random Forest

In [124]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(max_depth=5, random_state=0, criterion='gini')
decision_tree.fit(x_train_vectorised_cv, y_train_email)

y_pred = decision_tree.predict(x_test_vectorised_cv)

print(f'Matrix:\n{confusion_matrix(y_test_email, y_pred)}\nClassification Report:\n{classification_report(y_test_email, y_pred)}\nAccuracy: {accuracy_score(y_test_email, y_pred)}')

from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=5, random_state=0, criterion='gini', n_estimators=100)
random_forest.fit(x_train_vectorised_tfidf, y_train_email)

y_pred = random_forest.predict(x_test_vectorised_tfidf)

print(f'Matrix:\n{confusion_matrix(y_test_email, y_pred)}\nClassification Report:\n{classification_report(y_test_email, y_pred)}\nAccuracy: {accuracy_score(y_test_email, y_pred)}')

ValueError: Number of labels=1356 does not match number of samples=14907

## Dataset #2: Access behavior anomaly

In [121]:
from sklearn.preprocessing import StandardScaler

df_3 = pd.read_csv('supervised_dataset.csv') # https://www.kaggle.com/datasets/tangodelta/api-access-behaviour-anomaly-dataset?select=supervised_dataset.csv

df_3.drop(df_3.columns[0], inplace=True, axis=1)
df_3.drop('_id', inplace=True, axis=1)
df_3.drop('source', inplace=True, axis=1)
df_3.dropna(inplace=True)

le = LabelEncoder()
df_3['ip_type'] = le.fit_transform(df_3['ip_type'])
df_3['classification'] = le.fit_transform(df_3['classification'])
x = df_3.drop('classification', axis=1)
y = df_3['classification']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
scaler.fit(x_train)

scaled_x_train = scaler.transform(x_train)
scaled_x_test = scaler.transform(x_test)

## Decision Tree and Random Forest

In [123]:
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()

decision_tree.fit(scaled_x_train, y_train)
y_pred = decision_tree.predict(scaled_x_test)

print(f'Matrix:\n{confusion_matrix(y_test, y_pred)}\nClassification Report:\n{classification_report(y_test, y_pred)}\nAccuracy: {accuracy_score(y_test, y_pred)}')

random_forest.fit(scaled_x_train, y_train)
y_pred = random_forest.predict(scaled_x_test)

print(f'Matrix:\n{confusion_matrix(y_test, y_pred)}\nClassification Report:\n{classification_report(y_test, y_pred)}\nAccuracy: {accuracy_score(y_test, y_pred)}')

Matrix:
[[218   0]
 [  0 121]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       218
           1       1.00      1.00      1.00       121

    accuracy                           1.00       339
   macro avg       1.00      1.00      1.00       339
weighted avg       1.00      1.00      1.00       339

Accuracy: 1.0
Matrix:
[[218   0]
 [  0 121]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       218
           1       1.00      1.00      1.00       121

    accuracy                           1.00       339
   macro avg       1.00      1.00      1.00       339
weighted avg       1.00      1.00      1.00       339

Accuracy: 1.0
